In [5]:
import cv2
import numpy as np

In [22]:
video = cv2.VideoCapture(r'C:\Users\kapitoo\Downloads\feter_meshaltet.mp4')

# Get the video properties (width, height, frames per second)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Adjust the codec as needed
out = cv2.VideoWriter(r'C:\Users\kapitoo\Downloads\output_video.avi', fourcc, fps, (width, height))

while True:
    ret, orig_frame = video.read()

    if not ret:
        video = cv2.VideoCapture(r'C:\Users\kapitoo\Downloads\feter_meshaltet.mp4')
        continue

    frame = cv2.GaussianBlur(orig_frame, (5, 5), 0)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Adjusted HSV values for yellow
    low_yellow = np.array([15, 50, 150])
    up_yellow = np.array([35, 255, 255])

    # Adjusted HSV values for white
    low_white = np.array([0, 0, 200])
    up_white = np.array([255, 30, 255])

    # Create masks for yellow and white
    mask_yellow = cv2.inRange(hsv, low_yellow, up_yellow)
    mask_white = cv2.inRange(hsv, low_white, up_white)

    # Combine the masks
    combined_mask = cv2.bitwise_or(mask_yellow, mask_white)

    # Set the region of interest (ROI) to a slightly lower portion of the image
    height, width = frame.shape[:2]
    roi_mask = np.zeros_like(combined_mask)
    roi_mask[int(0.6 * height):height, :] = 255
    combined_mask = cv2.bitwise_and(combined_mask, roi_mask)

    edges = cv2.Canny(combined_mask, 50, 150)  # Adjust the thresholds

    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=100)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)

    # Write the frame to the output video
    out.write(frame)

    cv2.imshow("frame", frame)
    cv2.imshow("edges", edges)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Release the VideoWriter and VideoCapture objects
out.release()
video.release()
cv2.destroyAllWindows()
